In [ ]:
!apt install libeigen3-dev
!pip install MultiScaleOT

In [ ]:
import  numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold, datasets

# generate two datsets
def generateHeart(num_points):

  #source
  #https://medium.com/the-stata-guide/i-stata-391736dc6bad

  #t = np.linspace(0, np.pi*2, num_points+1)[:-1]

  t = 2*np.pi * np.random.random_sample(num_points)
  x = np.sin(t)**3  #+ np.random.normal(0,0.05, num_points)
  y = 13*np.cos(t)-5*np.cos(2*t)-2*np.cos(3*t)-np.cos(4*t) #+ np.random.normal(0,0.05, num_points)

  # rescale
  y += (np.abs(np.min(y)) - np.max(y))/2
  y /= np.max(y)

  # x points, y points and colors
  return np.array([x,y,t], dtype=np.double).T


def generateCircle(num_points):
  t = 2*np.pi * np.random.random_sample(num_points)
  #t = np.linspace(0, np.pi*2, num_points+1)[:-1]
  x = np.sin(t) #+ 2#+ np.random.normal(0,0.05, num_points)
  y = np.cos(t) #+ np.random.normal(0,0.05, num_points)

  return np.array([x,y,t], dtype=np.double).T

def generateLine(num_points):
  x = 2 * np.random.random_sample(num_points) - 1
  #x = np.linspace(-1, 1, num_points)
  y = np.zeros(num_points) #+ np.random.normal(0,0.05, num_points)

  return np.array([x,y,x]).T

def generateSwirl(num_points):

  data =  datasets.make_swiss_roll(n_samples = num_points)
  return np.hstack((np.hstack((data[0][:,:1], data[0][:,2:])), np.reshape(data[1], (num_points,1))))

def generateCircles(num_points):
  t = 2*np.pi * np.random.random_sample(num_points)
  #t = np.linspace(0, np.pi*2, num_points+1)[:-1]
  x = np.sin(t) #+ 2#+ np.random.normal(0,0.05, num_points)
  y = np.cos(t) #+ np.random.normal(0,0.05, num_points)
  half = int(num_points/2)
  x[:half] = x[:half]*2
  y[:half] = y[:half]*2
  return np.array([x,y,t]).T


def plot(X):

  fig, ax = plt.subplots(figsize=(6, 6))
  #ax.set(xlim=(-1.3, 1.3), ylim=(-1.3, 1.3))
  ax.scatter(X[:,0],X[:,1], c=X[:,2], s=20.0)

n = 100
heart = generateHeart(n)
circle = generateCircle(n)
#line = generateLine(n)
#swirl = generateSwirl(n)
#circles = generateCircles(n)

h_c = heart[:,2:]
c_c = circle[:,2:]
#l_c = line[:,2:]
plot(heart)
plot(circle)
#plot(line)
#plot(swirl)
#plot(circles)
#np.savetxt("hearts.txt", hearts[:,:2].astype('float32'), header=(str(n)))
#np.savetxt("circles.txt", circles[:,:2].astype('float32'), header=(str(n)))

#X = np.genfromtxt("outhearts.txt", dtype='float')
#print(X)
#plot(np.hstack((X,hearts_c)))


In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse
import MultiScaleOT

hierarchyDepth=6
nLayers=hierarchyDepth+1
# create two point clouds: one sphere, one torus
#n = 10000
#pos1=sampleSphere(n)
#pos2=sampleTorus(n)

a=np.full(n,1/n,dtype=np.double)
b=np.full(n,1/n,dtype=np.double)

# generate multi-scale representations
MultiScaleSetup=MultiScaleOT.TMultiScaleSetup(np.ascontiguousarray(heart[:,:2]),a,hierarchyDepth,childMode=0)
MultiScaleSetup3=MultiScaleOT.TMultiScaleSetup(np.ascontiguousarray(circle[:,:2]),b,hierarchyDepth,childMode=0)
#MultiScaleSetup1=MultiScaleOT.TMultiScaleSetup(pos1,a,hierarchyDepth,childMode=0)
#MultiScaleSetup2=MultiScaleOT.TMultiScaleSetup(pos2,b,hierarchyDepth,childMode=0)
print("here")
# generate a cost function object
costFunction=MultiScaleOT.THierarchicalCostFunctionProvider_SquaredEuclidean(
        MultiScaleSetup,MultiScaleSetup3)
# eps scaling
epsScalingHandler=MultiScaleOT.TEpsScalingHandler()
epsScalingHandler.setupGeometricMultiLayerB(nLayers,.3E-3,4.,2,2)

# error goal
errorGoal=1E-3
# sinkhorn solver object
SinkhornSolver=MultiScaleOT.TSinkhornSolverStandard(epsScalingHandler,
        0,hierarchyDepth,errorGoal,
        MultiScaleSetup,MultiScaleSetup3,costFunction
        )

SinkhornSolver.initialize()
SinkhornSolver.solve()
